## Preprocessing

In [ ]:
# Importing the necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

# Read the data from charity_data.csv
url = "https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv"
df = pd.read_csv(url)

# Display the first few rows of the dataframe
df.head()

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df.drop(columns=["EIN", "NAME"], inplace=True)

In [ ]:
# Determine the number of unique values in each column.
unique_counts = df.nunique()
print(unique_counts)

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
app_counts = df["APPLICATION_TYPE"].value_counts()
print(app_counts)

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
app_cutoff = 500
application_types_to_replace = list(app_counts[app_counts < app_cutoff].index)

# Replace in the dataframe
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app, "Other")

# Check to make sure binning was successful
print(df['APPLICATION_TYPE'].value_counts())

In [ ]:
# Look at CLASSIFICATION value counts for binning
class_counts = df["CLASSIFICATION"].value_counts()
print(class_counts)

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts_gt1 = df["CLASSIFICATION"].value_counts()[df["CLASSIFICATION"].value_counts() > 1]
print(class_counts_gt1)

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
class_cutoff = 1000
classifications_to_replace = list(class_counts_gt1[class_counts_gt1 < class_cutoff].index)
# Replace in the dataframe
for cls in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls, "Other")

# Check to make sure binning was successful
print(df['CLASSIFICATION'].value_counts())

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
df = pd.get_dummies(df)

In [ ]:
# Split our preprocessed data into our features and target arrays
X = df.drop(columns=["IS_SUCCESSFUL"])
y = df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_optimized = tf.keras.models.Sequential()

# First hidden layer
nn_optimized.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=len(X_train_scaled[0])))

# Second hidden layer
nn_optimized.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn_optimized.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_optimized.summary()

In [ ]:
# Compile the model
nn_optimized.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn_optimized.fit(X_train_scaled, y_train, epochs=50, verbose=1)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_optimized.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file
nn_optimized.save("AlphabetSoupCharity_Optimization.h5")